# Hyperparameter Tuning on the Avila Pattern Classifier

The Avila dataset has been extracted from 800 images of the Avila Bible, a giant 12th-century Latin copy of the Bible. The dataset consists of various features about the images of the text, such as intercolumnar distance and margins of the text. The dataset also contains a class label that indicates if the pattern of the image falls into the most frequently occurring category or not. In this activity, you will build a Keras model similar to those in the previous activities, but this time, you will add regularization methods to your model as well. Then, you will use scikit-learn optimizers to perform tuning on the model hyperparameters, including the hyperparameters of the regularizers. Here are the steps you need to complete in this activity:

### 1. Load the dataset from the data subfolder of the Chapter05 folder from GitHub using X = pd.read_csv('../data/avila-tr_feats.csv') and y = pd.read_csv('../data/avila-tr_target.csv').

In [2]:
import pandas as pd 
import numpy as np 
from tensorflow import random

In [3]:
X = pd.read_csv('../data/avila-tr_feats.csv')
y = pd.read_csv('../data/avila-tr_target.csv')
print(X.shape, y.shape)

(10430, 10) (10430, 1)


### 2. Define a function that returns a Keras model with three hidden layers, the first of size 10, the second of size 6, and the third of size 4, all with L2 weight regularizations. Use these values as the hyperparameters for your model: activation='relu', loss='binary_crossentropy', optimizer='sgd', and metrics=\['accuracy'\]. Also, make sure to pass the L2 lambda hyperparameter as an argument to your function so that we can tune it later.

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l2

In [5]:
def build_model(l2_lambda):
    
    model = Sequential()
    model.add(Dense(10, input_dim=X.shape[1], activation='relu', kernel_regularizer=l2(l2_lambda)))
    model.add(Dense(6, activation='relu', kernel_regularizer=l2(l2_lambda)))
    model.add(Dense(4, activation='relu', kernel_regularizer=l2(l2_lambda)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='sgd',
        loss='binary_crossentropy',
        metrics=['accuracy',]
    )

    return model

### 3. Create the wrapper for your Keras model and perform GridSearchCV() on it using cv=5. Then, add the following values in the parameter grid: lambda_parameter = \[0.01, 0.5, 1\], epochs = \[50, 100\], and batch_size = \[20\]. This might take some time to process. Once the parameter search is complete, print the accuracy and the hyperparameters of the best cross-validation score. You can also print every other cross-validation score, along with the hyperparameters that resulted in that score.

In [6]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [7]:
SEED = 1

np.random.seed(SEED)
random.set_seed(SEED)

# keras wrapper
model = KerasClassifier(build_fn=build_model, verbose=1)

# hyperparameters for grid search
l2_lambda = [0.01, 0.5, 1]
epochs = [50, 100]
batch_size = [20]
param_grid = dict(l2_lambda=l2_lambda, epochs=epochs, batch_size= batch_size)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    verbose=1
)

results_1 = grid_search.fit(X, y)

6758 - accuracy: 0.5930
Epoch 66/100
8344/8344 [==============================] - 1s 112us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 67/100
8344/8344 [==============================] - 1s 130us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 68/100
8344/8344 [==============================] - 1s 103us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 69/100
8344/8344 [==============================] - 1s 109us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 70/100
8344/8344 [==============================] - 1s 108us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 71/100
8344/8344 [==============================] - 1s 107us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 72/100
8344/8344 [==============================] - 1s 121us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 73/100
8344/8344 [==============================] - 1s 115us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 74/100
8344/8344 [==============================] - 1s 115us/step - loss: 0.6758 - accuracy: 0.5930
Epoch 75/100
8344/8344

In [8]:
# print the results for best cross validation score
print("Best cross validation score =", results_1.best_score_)
print("Parameters for Best cross validation score=", results_1.best_params_)

# print the results for all evaluated hyperparameter combinations
accuracy_means = results_1.cv_results_['mean_test_score']
accuracy_stds = results_1.cv_results_['std_test_score']
parameters = results_1.cv_results_['params']
for p in range(len(parameters)):
    print("Accuracy %f (std %f) for params %r" % (accuracy_means[p], accuracy_stds[p], parameters[p]))

Best cross validation score = 0.7700862884521484
Parameters for Best cross validation score= {'batch_size': 20, 'epochs': 50, 'l2_lambda': 0.01}
Accuracy 0.770086 (std 0.005257) for params {'batch_size': 20, 'epochs': 50, 'l2_lambda': 0.01}
Accuracy 0.589070 (std 0.008244) for params {'batch_size': 20, 'epochs': 50, 'l2_lambda': 0.5}
Accuracy 0.589070 (std 0.008244) for params {'batch_size': 20, 'epochs': 50, 'l2_lambda': 1}
Accuracy 0.764621 (std 0.016414) for params {'batch_size': 20, 'epochs': 100, 'l2_lambda': 0.01}
Accuracy 0.589070 (std 0.008244) for params {'batch_size': 20, 'epochs': 100, 'l2_lambda': 0.5}
Accuracy 0.589070 (std 0.008244) for params {'batch_size': 20, 'epochs': 100, 'l2_lambda': 1}


### 4. Repeat the previous step, this time using GridSearchCV() on a narrower range with lambda_parameter = \[0.001, 0.01, 0.05, 0.1\], epochs = \[400\], and batch_size = \[10\]. It might take some time to process.

In [9]:
np.random.seed(SEED)
random.set_seed(SEED)

# keras wrapper
model = KerasClassifier(build_fn=build_model, verbose=1)

# hyperparameters for grid search
l2_lambda = [0.001, 0.01, 0.05, 0.1]
epochs = [400]
batch_size = [10]
param_grid = dict(l2_lambda=l2_lambda, epochs=epochs, batch_size= batch_size)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    verbose=1
)

results_2 = grid_search.fit(X, y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Epoch 1/400
8344/8344 [==============================] - 2s 257us/step - loss: 0.7031 - accuracy: 0.5756
Epoch 2/400
8344/8344 [==============================] - 2s 184us/step - loss: 0.6937 - accuracy: 0.5932
Epoch 3/400
8344/8344 [==============================] - 1s 172us/step - loss: 0.6872 - accuracy: 0.5961
Epoch 4/400
8344/8344 [==============================] - 2s 181us/step - loss: 0.6725 - accuracy: 0.6107
Epoch 5/400
8344/8344 [==============================] - 2s 201us/step - loss: 0.6456 - accuracy: 0.6547
Epoch 6/400
8344/8344 [==============================] - 1s 172us/step - loss: 0.6099 - accuracy: 0.7049
Epoch 7/400
8344/8344 [==============================] - 1s 174us/step - loss: 0.5821 - accuracy: 0.7239
Epoch 8/400
8344/8344 [==============================] - 2s 197us/step - loss: 0.5614 - accuracy: 0.7386
Epoch 9/400
8344/83

KeyboardInterrupt: 

### 5. Repeat the previous step, but remove the L2 regularizers from your Keras model and instead of adding dropout regularization with the rate parameter at each hidden layer. Perform GridSearchCV() on the model using the following values in the parameter grid and print the results: rate = \[0, 0.2, 0.4\], epochs = \[350, 400\], and batch_size = \[10\].

In [10]:
from keras.layers import Dropout

def build_model2(dropout_rate):

    model = Sequential()
    model.add(Dense(10, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate, seed=SEED))
    model.add(Dense(6, activation='relu'))
    model.add(Dropout(dropout_rate, seed=SEED))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(dropout_rate, seed=SEED))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer='sgd',
        loss='binary_crossentropy',
        metrics=['accuracy',]
    )

    return model

In [12]:
np.random.seed(SEED)
random.set_seed(SEED)

# keras wrapper
model = KerasClassifier(build_fn=build_model2, verbose=1)

# hyperparameters for grid search
dropout_rate = [0, 0.2, 0.4]
epochs = [350, 400]
batch_size = [10]
param_grid = dict(dropout_rate=dropout_rate, epochs=epochs, batch_size= batch_size)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    verbose=1
)

results_3 = grid_search.fit(X, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Epoch 1/350
8344/8344 [==============================] - 3s 336us/step - loss: 0.6820 - accuracy: 0.5756
Epoch 2/350
8344/8344 [==============================] - 2s 253us/step - loss: 0.6735 - accuracy: 0.5941
Epoch 3/350
8344/8344 [==============================] - 2s 230us/step - loss: 0.6686 - accuracy: 0.5960
Epoch 4/350
8344/8344 [==============================] - 2s 187us/step - loss: 0.6542 - accuracy: 0.6104
Epoch 5/350
8344/8344 [==============================] - 2s 211us/step - loss: 0.6267 - accuracy: 0.6540
Epoch 6/350
8344/8344 [==============================] - 2s 210us/step - loss: 0.5886 - accuracy: 0.7047
Epoch 7/350
8344/8344 [==============================] - 2s 181us/step - loss: 0.5574 - accuracy: 0.7293
Epoch 8/350
8344/8344 [==============================] - 2s 182us/step - loss: 0.5350 - accuracy: 0.7426
Epoch 9/350
8344/83

KeyboardInterrupt: 